In [1]:
import folium
import pandas as pd
import os
import time
from datetime import datetime
import numpy as np


import plotly.express as px
!pip install folium matplotlib mapclassify
!pip install geopandas

import geopandas as gpd
import shapely.geometry
import folium

In [2]:
#df = pd.read_csv('Residential_dwellings.csv')
df = pd.read_csv('https://data.melbourne.vic.gov.au/api/views/rm92-h5tq/rows.csv?accessType=DOWNLOAD')
df

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Dwelling type,Dwelling number,x coordinate,y coordinate
0,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential Apartments,196,144.95651,-37.82098
1,2020,1,611395,611395,561-581 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential Apartments,189,144.95591,-37.82109
2,2020,11,103957,103957,517-537 Flinders Lane MELBOURNE VIC 3000,Melbourne (CBD),Residential Apartments,26,144.95666,-37.81987
3,2020,11,103986,103986,556-560 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential Apartments,10,144.95577,-37.82061
4,2020,11,103987,103987,550-554 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential Apartments,176,144.95597,-37.82039
...,...,...,...,...,...,...,...,...,...,...
10397,2020,2547,615563,615563,89 Barnett Street KENSINGTON VIC 3031,Kensington,House/Townhouse,1,144.93220,-37.79404
10398,2020,2547,615570,615570,91 Barnett Street KENSINGTON VIC 3031,Kensington,House/Townhouse,1,144.93219,-37.79410
10399,2020,2547,615573,615573,93 Barnett Street KENSINGTON VIC 3031,Kensington,House/Townhouse,1,144.93218,-37.79415
10400,2020,2547,615577,615577,95 Barnett Street KENSINGTON VIC 3031,Kensington,House/Townhouse,1,144.93217,-37.79420


Showing the map of Melbourne

In [3]:
m = folium.Map([-37.8,145],zoom_start=11)
m

Grouping buildings by block ID and CLUE small area and aggregating on Dwelling number.

In [5]:
groupbyfields = ['CLUE small area','Block ID']
aggregatebyfields = {'Dwelling number': ["sum"],
                    'y coordinate':["first"],
                     'x coordinate':["first"]}

dwellingsByLocn = pd.DataFrame(df.groupby(groupbyfields, as_index=False).agg(aggregatebyfields))
dwellingsByLocn.columns = dwellingsByLocn.columns.get_level_values(0)
dwellingsByLocn

,CLUE small area,Block ID,Dwelling number,y coordinate,x coordinate
0,Carlton,201,250,-37.79430,144.96594
1,Carlton,202,47,-37.79568,144.96601
2,Carlton,203,81,-37.79681,144.96582
3,Carlton,204,567,-37.79783,144.96517
4,Carlton,205,84,-37.79900,144.96476
...,...,...,...,...,...
445,West Melbourne (Residential),439,25,-37.80883,144.95290
446,West Melbourne (Residential),440,519,-37.80952,144.95061
447,West Melbourne (Residential),441,18,-37.80972,144.94907
448,West Melbourne (Residential),443,84,-37.81055,144.94989


Plotting data on high rise dwellings in Melbourne city. Size of bubble denotes the number of residents. 

In [7]:
fig = px.scatter_mapbox(dwellingsByLocn, lat="y coordinate", lon="x coordinate",
                        hover_name="CLUE small area",
                        hover_data=["CLUE small area", "Block ID"],
                        color_continuous_scale=px.colors.sequential.Plasma, zoom=12, height=600,
                        color="Dwelling number",
                        size = "Dwelling number"
                       )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Extracting data for busstops in Melbourne

In [38]:
bus_stops_df = pd.read_csv("https://data.melbourne.vic.gov.au/api/views/vzsu-xnf6/rows.csv?accessType=DOWNLOAD")
bus_stops_df = bus_stops_df[['the_geom','OBJECTID']]
bus_stops_df['Longitude'] = bus_stops_df['the_geom'].apply(lambda s: s.split()[1][1:]).astype(float)
bus_stops_df['Latitude'] = bus_stops_df['the_geom'].apply(lambda s: s.split()[2][:-1]).astype(float)
bus_stops_df = bus_stops_df.drop('the_geom',axis=1)
bus_stops_df

,OBJECTID,Longitude,Latitude
0,303,144.921856,-37.787445
1,355,144.932393,-37.803842
2,434,144.963813,-37.816247
3,519,144.951340,-37.796769
4,600,144.958179,-37.815487
...,...,...,...
304,44103,144.961272,-37.822043
305,44125,144.918225,-37.784414
306,44170,144.984817,-37.846370
307,44287,144.961123,-37.810347


In [39]:


rest_VIC = gpd.read_file(
    "https://data.melbourne.vic.gov.au/api/geospatial/ss79-v558?method=export&format=GeoJSON"
)

fig = px.scatter_mapbox(bus_stops_df, lat="Latitude", lon="Longitude",
                        hover_name="OBJECTID",
                        color_continuous_scale=px.colors.sequential.Plasma, zoom=12, height=600,
                       )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})



fig.show()


Transport data source: http://data.ptv.vic.gov.au/downloads/gtfs.zip

# Action Plan
* Get data on transportation systems in melbourne
* Plot them together with high rise data
* Find the closest transportation point to each high rise dwelling
* Calculate the average distance between a dwelling and a transportation point. 
* Plot these distances on the map to see how each district performs

## Next Analysis:
* Developing buildings vs transportation points

## Secondary Analysis:
* Availability of public transport vs median income(demographic data)
* High rise buildings vs median income(demographic data)

## Livability:
* Livability score vs transportation points